In [40]:
import matplotlib as plt
import seaborn as sns
from typing import Callable
import pandas as pd

In [54]:
params = {
    "steps": 10000
}

In [69]:
def euler(f: Callable, condition: tuple, final_x: float, steps=None, h=None):
    if not steps is None:
        h = (final_x - condition[0]) / steps
    elif h is None:
        h = (final_x - condition[0]) / params["steps"]
    cur_x = condition[0]
    cur_y = condition[1]
    while cur_x < final_x-h:
        cur_y = cur_y + h*f(cur_x,cur_y)
        cur_x +=h
    return cur_y

def euler_retun_all(f: Callable, condition: tuple, final_x: float, steps=None, h=None):
    if not steps is None:
        h = (final_x - condition[0]) / steps
    elif h is None:
        h = (final_x - condition[0]) / params["steps"]
    out_x, out_y = [],[]
    out_x.append(condition[0])
    out_y.append(condition[1])
    while out_x[-1] < final_x:
        out_y.append(out_y[-1] + h*f(out_x[-1],out_y[-1]))
        out_x.append(out_x[-1]+h)
    return pd.DataFrame({"x":out_x,"y":out_y})

In [103]:
problem_1 = euler_retun_all(lambda x,y: 2*x - 3*y + 1, (1,5), 1.2, h=0.1)
print(problem_1)
problem_2 = euler_retun_all(lambda x,y: x + y**2, (0,0), 0.2, h=0.1)
print(problem_2)
problem_3 = euler_retun_all(lambda x,y: 2*x - 3*y + 1, (1,5), 1.2, h=0.05)
print(problem_3)
problem_4 = euler_retun_all(lambda x,y: x + y**2, (0,0), 0.2, h=0.05)
print(problem_4)

     x     y
0  1.0  5.00
1  1.1  3.80
2  1.2  2.98
     x     y
0  0.0  0.00
1  0.1  0.00
2  0.2  0.01
      x         y
0  1.00  5.000000
1  1.05  4.400000
2  1.10  3.895000
3  1.15  3.470750
4  1.20  3.115138
      x         y
0  0.00  0.000000
1  0.05  0.000000
2  0.10  0.002500
3  0.15  0.007500
4  0.20  0.015003


4.466049527037922